In [1]:
#creates a dictionary containing sentence embeddings (averaged word embeddings)

In [2]:
import sklearn
from sklearn.neighbors import NearestNeighbors
import nltk
import numpy as np
import pickle
from nltk.tokenize import word_tokenize
from tqdm import tqdm

In [3]:
sent_id_dict = pickle.load(open("sent_id_dict.p", "rb"))

In [4]:
embed_dict = pickle.load(open ("pubmed_embed_dict.p", "rb"))

In [5]:
embedding_len = 200

In [6]:
X = np.zeros((len(sent_id_dict),embedding_len), dtype=float)
#print(len(sent_id_dict))
print(X.shape)
print(X)

(40822, 200)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [7]:
UNK = np.random.uniform(low=-1.0, high=1.0, size=(200,) )
NUM = np.random.uniform(low=-1.0, high=1.0, size=(200,) )
print(embed_dict['.'].shape)
print(UNK.shape)
print(NUM.shape)
embed_dict['UNK'] = UNK
embed_dict['-UNK-'] = UNK
embed_dict['NUM'] = NUM

(200,)
(200,)
(200,)


In [9]:
tok_sent_list = []
for sent in tqdm(sent_id_dict):
    tok_sent = word_tokenize(sent)
    new_tok_sent = []
    for word in tok_sent: 
        if word.isdigit():
            #print(word)
            new_tok_sent.append('NUM')
        elif word in embed_dict:
            new_tok_sent.append(word)
        else:
            new_tok_sent.append('UNK')
    tok_sent_list.append(new_tok_sent)

100%|██████████| 40822/40822 [00:06<00:00, 6557.11it/s]


In [16]:
print(len(tok_sent_list))

40822


In [10]:
encoded_sent_id_dict = {}
sent_embed_list = []
for sent in tqdm(tok_sent_list):
    index = tok_sent_list.index(sent)
    sent_embed = np.zeros(embedding_len)

    for word in sent:
        word_embed = embed_dict[word]
        sent_embed = sent_embed + word_embed
    sent_embed = sent_embed/len(tok_sent)
    X[index] = X[index] + sent_embed
    string = ' '.join(str(x) for x in sent_embed)
    sent_embed_list.append(sent_embed)
    encoded_sent_id_dict[string] = sent

100%|██████████| 40822/40822 [00:22<00:00, 1835.37it/s]


In [23]:
pickle.dump(encoded_sent_id_dict, open ("encoded_sent_id_dict.p", "wb"))
pickle.dump(sent_embed_list, open ("sent_embed_list.p", "wb"))
pickle.dump(tok_sent_list, open ("tok_sent_list.p", "wb"))